# Problem Set #10
## Tc2006 Programming Languages

May 17, 2021.

_Authors of this notebook’s solution:_

- _A01226707 Emilio Canton_
- _A01654203 Jorge Palacios_
- _Mauricio Cassab A01025475_

**Instructions:** Write the Clojure code to solve the following problems. Make sure each function passes all the unit tests.

In [4]:
;;; External function required for this notebook.
(require '[clojure.test :refer [is]])

nil

---

## Problem 1

Write a macro called `$if`. Its purpose is to provide a conditional statement that is syntactically a bit more similar to those found in languages like Pascal or Fortran. It has the following form:

> (`$if` $condition$ `:then` $exp_1$ $exp_2$ $\ldots$ `:else` $exp_3$ $exp_4$ $\ldots$)

It should basically expand to the following conventional Clojure if form:

> (`if` $condition$ (`do` $exp_1$ $exp_2$ $\ldots$) (`do` $exp_3$ $exp_4$ $\ldots$))

Almost everything is optional in the `$if` macro, except $condition$. Also, the `:else` keyword may come before the `:then` keyword. The following unit tests show some legal forms and their results:

In [14]:
(defn extract-delimited
    [start end args]
    (->> args
        (drop-while #(not= % start))
        rest 
        (take-while #(not= % end))))

(defmacro $if
  [condition & args]
   `(if ~condition 
        (do ~@(extract-delimited :then :else args))
        (do ~@(extract-delimited :else :then args))))

#'user/$if

In [15]:
;;; Unit tests:
(is (= 'ok ($if (> 3 1) :then 'ok :else 'oops)))
(is (= 'oops ($if (< 3 1) :then 'ok :else 'oops)))
(is (= 'ok ($if (> 3 1) :then 'ok)))
(is (= 'ok ($if (< 3 1) :else 'ok)))
(is (nil? ($if (> 3 1) :else 'oops)))
(is (nil? ($if (> 3 1) :then)))
(is (nil? ($if (> 3 1))))
(is (= "Then section."
       (with-out-str
         ($if (> 3 1)
              :else (print "Else section.") 'oops
              :then (print "Then section.") 'ok))))
(is (= "Else section."
       (with-out-str
         ($if (< 3 1)
              :else (print "Else section.") 'oops
              :then (print "Then section.") 'ok))))

true

---

## Problem 2

Write a macro called `def-pred`, that takes a $name$, an $arg$ vector, and a $body$ of one or more expressions. The macro should define two predicate functions: a regular one and its negated version. The name of the negated predicate should be the same as $name$ but with a “`not-`” prefix, and its result should be negated using the `not` function. An example:

    (def-pred less-than-one? [x] (< x 1)))
    
macro expands to:

    (do (defn less-than-one? [x] (do (< x 1)))
        (defn not-less-than-one? [x] (not (do (< x 1)))))

In [7]:
(defmacro def-pred
  ([condition & args]
    `(do ~@(map (fn [[x y]] `(def ~x ~y)) ))))

#'user/def-pred

In [5]:
;;; Unit tests:
(def-pred less-than-one? [x] (< x 1))
(is (less-than-one? 0))
(is (not-less-than-one? 1))
(is (not (less-than-one? 1)))
(is (not (not-less-than-one? 0)))
(def-pred plural-1? [s] (= \s (last s)))
(is (plural-1? "boys"))
(is (not (plural-1? "girl")))
(is (not (not-plural-1? "boys"))
(is (not-plural-1? "girl")))
(def-pred plural-2? [s] (print "check s in" s) (= \s (last s)))
(is (= "check s in boys")
    (with-out-str (plural-2? "boys")))
(is (= "check s in girls")
    (with-out-str (plural-2? "girl")))
(is (= "check s in boys")
    (with-out-str (not-plural-2? "boys")))
(is (= "check s in girls")
    (with-out-str (not-plural-2? "girl")))

Execution error (ArityException) at nrepl.middleware.interruptible-eval/evaluate (interruptible_eval.clj:79).
Wrong number of args (3) passed to: user/def-pred
Syntax error compiling at (REPL:3:5).
Unable to resolve symbol: less-than-one? in this context
Syntax error compiling at (REPL:4:5).
Unable to resolve symbol: not-less-than-one? in this context
Syntax error compiling at (REPL:5:10).
Unable to resolve symbol: less-than-one? in this context
Syntax error compiling at (REPL:6:10).
Unable to resolve symbol: not-less-than-one? in this context
Execution error (ArityException) at nrepl.middleware.interruptible-eval/evaluate (interruptible_eval.clj:79).
Wrong number of args (3) passed to: user/def-pred
Syntax error compiling at (REPL:8:5).
Unable to resolve symbol: plural-1? in this context
Syntax error compiling at (REPL:9:10).
Unable to resolve symbol: plural-1? in this context
Syntax error compiling at (REPL:10:10).
Unable to resolve symbol: not-plural-1? in this context
Execution err

class clojure.lang.Compiler$CompilerException: 

---

## Problem 3

Write a Clojure macro called `def-many`. This macro allows defining many global bindings in one place. It has the following form:

$$
(\texttt{def-many} \; \textit{var}_1 \; \textit{expr}_1 \; \textit{var}_2 \; \textit{expr}_2 \; \ldots \; \textit{var}_n \; \textit{expr}_n)
$$

Where every $\textit{var}_i$ is a symbol and every $\textit{expr}_i$ is an arbitrary expression. The macro evaluates $\textit{expr}_1$ and binds the result to $\textit{var}_1$ (using the `def` special form), then evaluates $\textit{expr}_2$ and binds the result to $\textit{var}_2$, and so on.

The macro expands to the following form:

$$
(\texttt{do} \; (\texttt{def} \; \textit{var}_1 \; \textit{expr}_1) \;
  (\texttt{def} \; \textit{var}_2 \; \textit{expr}_2)
  \cdots
  (\texttt{def} \; \textit{var}_n \; \textit{expr}_n))
$$

In [41]:
(defmacro def-many
    ([] nil)
  ([& args]
    `(do ~@(map (fn [[x y]] `(def ~x ~y)) (partition 2 args)))))

#'user/def-many

In [43]:
;;; Unit tests:
(def-many)
(def-many x (* (+ 3 3) (/ 21 3)))
(is (= x 42))
(def-many a (+ 1 2)
          b (* 2 a)
          c (/ (inc b) a))
(is (= 34/3 (+ a b c)))
(def-many alpha "beta"
          gamma "delta"
          epsilon "zeta"
          eta "theta"
          iota "kappa")
(is (= "betadeltazetathetakappa"
       (str alpha gamma epsilon eta iota)))

true